##### Copyright 2018 The TensorFlow Hub Authors.

##### Modifications Copyright 2019 Tomoaki Masuda.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Copyright 2018 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

このノートブックは、以下のノートブックを元に日本語訳、一部章立ての再構成、加筆を行いました。https://github.com/tensorflow/hub/blob/master/examples/colab/semantic_similarity_with_tf_hub_universal_encoder.ipynb


# Universal Sentence Encoderで話題が近い文を見分ける

このノートブックは、Universal Sentence Encoderを使って、文の類似度判定や分類のタスクに使う方法を説明します。 

 Universal Sentence Encoderを使用すると、従来個々の「単語」のembedding（単語の埋め込み、または単語の分散表現）を取得するのと同じくらい簡単に、「文」のembeddingを取得できます。文のembeddingを使って、文の意味の類似性を計算することや、少量の教師あり学習データセットを使い、下流の分類タスクでより良い精度を得ることができます。


## A. 環境を準備する

このセクションでは、TF Hub上のUniversal Sentence Encoderにアクセスするための環境を設定し、エンコーダを単語、文、および段落に適用する例を試します。


In [ ]:
# Install the latest Tensorflow version.
!pip3 install --quiet "tensorflow>=1.7"
# Install TF-Hub.
!pip3 install --quiet tensorflow-hub


 TensorFlowインストールの詳細情報は https://www.tensorflow.org/install/ にあります。 


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns


## B. データセットを準備する

## C. データセットを前処理する

学習済みモデルを使うため、データセットの準備は不要です。


## D. モデルを取得する

`https://tfhub.dev/google/universal-sentence-encoder/1` から、Universal Sentence Encoderの学習済みモジュールを取得します。

In [ ]:
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder/1")


## E. モデルを学習させる

学習済みモデルを使うため不要です。

## F. 学習済みモデルを評価する

### 1. 単語・文・パラグラフで分散表現を得る

可変長の入力に対応しているため、単語・文・パラグラフそれぞれで分散表現（埋め込み）が得られます。

In [ ]:
# Compute a representation for each message, showing various lengths supported.
word = "Elephant"
sentence = "I am a sentence for which I would like to get its embedding."
paragraph = (
    "Universal Sentence Encoder embeddings also support short paragraphs. "
    "There is no hard limit on how long the paragraph is. Roughly, the longer "
    "the more 'diluted' the embedding will be.")
messages = [word, sentence, paragraph]

# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  message_embeddings = session.run(embed(messages))

  for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
    print("Message: {}".format(messages[i]))
    print("Embedding size: {}".format(len(message_embedding)))
    message_embedding_snippet = ", ".join(
        (str(x) for x in message_embedding[:3]))
    print("Embedding: [{}, ...]\n".format(message_embedding_snippet))


### 2. 文意の類似度を判定する

Universal Sentence Encoderによって生成する文のembeddingは、ほぼ正規化されています。 2つの文の意味がどれだけ近いかは、エンコーディングの内積として計算できます。 


In [ ]:
def plot_similarity(labels, features, rotation):
  corr = np.inner(features, features)
  sns.set(font_scale=1.2)
  g = sns.heatmap(
      corr,
      xticklabels=labels,
      yticklabels=labels,
      vmin=0,
      vmax=1,
      cmap="YlOrRd")
  g.set_xticklabels(labels, rotation=rotation)
  g.set_title("Semantic Textual Similarity")


def run_and_plot(session_, input_tensor_, messages_, encoding_tensor):
  message_embeddings_ = session_.run(
      encoding_tensor, feed_dict={input_tensor_: messages})
  plot_similarity(messages_, message_embeddings_, 90)


### 3. 文章の類似度を可視化する

ヒートマップで類似度を可視化してみましょう。得られるグラフは9×9の行列です。各カラム `[i, j]`は、文 `i` と `j` それぞれのエンコーディングの内積に基づき、色付けされています。 


In [ ]:
messages = [
    # Smartphones
    "I like my phone",
    "My phone is not good.",
    "Your cellphone looks great.",

    # Weather
    "Will it snow tomorrow?",
    "Recently a lot of hurricanes have hit the US",
    "Global warming is real",

    # Food and health
    "An apple a day, keeps the doctors away",
    "Eating strawberries is healthy",
    "Is paleo better than keto?",

    # Asking about age
    "How old are you?",
    "what is your age?",
]

similarity_input_placeholder = tf.placeholder(tf.string, shape=(None))
similarity_message_encodings = embed(similarity_input_placeholder)
with tf.Session() as session:
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  run_and_plot(session, similarity_input_placeholder, messages,
               similarity_message_encodings)


## G. 発展: STSベンチマークで評価する


 [**STSベンチマーク**](http://ixa2.si.ehu.es/stswiki/index.php/STSbenchmark)を使うと、文の埋め込みを使って計算された類似度スコアが、どの程度人間の判断と一致するか評価できます。ベンチマークには、多様な文の組合せペアに対し、類似度スコアを返すシステムが必要です。次に、 [ピアソン相関](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient)を使って、人間の判断と比較した、マシンの類似度スコアの品質を評価します。 



### 1. データセットを準備する

データセットをダンロードします。


In [ ]:
import pandas
import scipy
import math


def load_sts_dataset(filename):
  # Loads a subset of the STS dataset into a DataFrame. In particular both
  # sentences and their human rated similarity score.
  sent_pairs = []
  with tf.gfile.GFile(filename, "r") as f:
    for line in f:
      ts = line.strip().split("\t")
      # (sent_1, sent_2, similarity_score)
      sent_pairs.append((ts[5], ts[6], float(ts[4])))
  return pandas.DataFrame(sent_pairs, columns=["sent_1", "sent_2", "sim"])


def download_and_load_sts_data():
  sts_dataset = tf.keras.utils.get_file(
      fname="Stsbenchmark.tar.gz",
      origin="http://ixa2.si.ehu.es/stswiki/images/4/48/Stsbenchmark.tar.gz",
      extract=True)

  sts_dev = load_sts_dataset(
      os.path.join(os.path.dirname(sts_dataset), "stsbenchmark", "sts-dev.csv"))
  sts_test = load_sts_dataset(
      os.path.join(
          os.path.dirname(sts_dataset), "stsbenchmark", "sts-test.csv"))

  return sts_dev, sts_test


sts_dev, sts_test = download_and_load_sts_data()


### 2. 評価グラフを作成する


In [ ]:
text_a = sts_dev['sent_1'].tolist()
text_b = sts_dev['sent_2'].tolist()
dev_scores = sts_dev['sim'].tolist()
sts_input1 = tf.placeholder(tf.string, shape=(None))
sts_input2 = tf.placeholder(tf.string, shape=(None))

# For evaluation we use exactly normalized rather than
# approximately normalized.
sts_encode1 = tf.nn.l2_normalize(embed(sts_input1))
sts_encode2 = tf.nn.l2_normalize(embed(sts_input2))
sim_scores = tf.reduce_sum(tf.multiply(sts_encode1, sts_encode2), axis=1)


### 3. 文のembeddingを評価する


In [ ]:
def run_sts_benchmark(session):
  """Returns the similarity scores"""
  emba, embb, scores = session.run(
      [sts_encode1, sts_encode2, sim_scores],
      feed_dict={
          sts_input1: text_a,
          sts_input2: text_b
      })
  return scores


with tf.Session() as session:
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  scores = run_sts_benchmark(session)

pearson_correlation = scipy.stats.pearsonr(scores, dev_scores)
print('Pearson correlation coefficient = {0}\np-value = {1}'.format(
    pearson_correlation[0], pearson_correlation[1]))